## Домашнее задание 5 ##

In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

**1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)**

In [4]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [5]:
#Удалим неинформативные признаки
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
df[['Geography', 'Gender']].describe()

,Geography,Gender
count,10000,10000
unique,3,2
top,France,Male
freq,5014,5457


In [9]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [10]:
#Проверим соотношение целевого признака в train и test
print(f'Доля целевого призака в train = {y_train.mean()}')
print(f'Доля целевого призака в test = {y_test.mean()}')

Доля целевого призака в train = 0.20373333333333332
Доля целевого призака в test = 0.2036


In [11]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [12]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [14]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [15]:
#Объединим
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [16]:
#Добавим модели
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [17]:
#Напишем функцию для расчета метрик
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix

def scores(model):
    

    pipeline = Pipeline([('features', feats), ('model', model)])
    pipeline.fit(X_train, y_train)

    preds = pipeline.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_true=y_test, y_score=preds)
    ix = np.argmax(fscore)
    
    cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
    print(cnf_matrix)
    
    return [round(i, 3) for i in
            [thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc]]



In [18]:
%%time

results = pd.DataFrame([
    scores(model=RandomForestClassifier(n_estimators = 100, random_state=3)),
    scores(model=GradientBoostingClassifier(random_state=3)),
    scores(model=LogisticRegression(solver = 'lbfgs', random_state=3))
], columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['RandomForestClassifier', 'GradientBoosting', 'LogisticRegression']
results = results.set_index('models')

[[1850  141]
 [ 194  315]]
[[1863  128]
 [ 206  303]]
[[1655  336]
 [ 221  288]]
Wall time: 1.33 s


**2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)**

In [19]:
results.sort_values(by='Precision', axis=0, ascending=False)

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,
GradientBoosting,0.409,0.646,0.704,0.597,0.876
RandomForestClassifier,0.400,0.655,0.684,0.629,0.861
LogisticRegression,0.290,0.510,0.462,0.568,0.772


На данном этапе, выбор метрики без расчетов в денежном эквиваленте не имеет смысла, т.к. затраты от бизнеса к бизнесу могут сильно различаться. Попробуем рассчитать экономическую эффективность на GradientBoosting т.к. у этой модели самое меньшее количество FP.

**3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.**

GradientBoostingClassifier TN = 1863, FP = 128, FN = 206, TP = 303. При затратах в 1 доллар на одного удержанного бизнес потратит 303 + 128 = 431 доллар, а получит 303*2 = 606. Итоговая прибыль 606 - 431 = 177 долларов.

RandomForest TN = 1850, FP = 141, FN = 194, TP = 315. При затратах в 1 доллар на одного удержанного бизнес потратит 315 + 141 = 456 долларов, а получит 315*2 = 630. Итоговая прибыль 630 - 456 = 174 доллара.

LogisticRegression TN = 1655, FP = 336, FN = 221, TP = 288. При затратах в 1 доллар на одного удержанного бизнес потратит 288 + 336 = 624 доллара, а получит 288*2 = 576. Итоговая прибыль (убыток) 576 - 624 = -48 долларов.

Итого прибыль больше при использовании модели GradientBoostingClassifier со средним результатом f1 и большим значением Precision.

**4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3**

In [27]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier()),
])

In [28]:
from sklearn.model_selection import GridSearchCV

params={'classifier__n_estimators': [20, 50, 100],
        'classifier__max_depth': [2, 3, 4, 6]
       }

In [35]:
grid = GridSearchCV(pipeline,
                    param_grid=params,
                    cv=6,
                    scoring = 'average_precision')

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 3, 'classifier__n_estimators': 100}

In [49]:
%%time

results = pd.DataFrame([
    scores(model=RandomForestClassifier(n_estimators = 100)),
    scores(model=GradientBoostingClassifier()),
    scores(model=LogisticRegression(solver = 'lbfgs')),
    scores(model=GradientBoostingClassifier(max_depth=3, n_estimators=100))
], columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['RandomForestClassifier', 'GradientBoosting', 'LogisticRegression', 'GradientBoosting_t']
results = results.set_index('models')

C:\Users\Coolver\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


[[1991    0]
 [ 509    0]]
[[1863  128]
 [ 206  303]]
[[1655  336]
 [ 221  288]]
[[1863  128]
 [ 206  303]]
Wall time: 2.06 s


In [50]:
results.sort_values(by='Precision', axis=0, ascending=False)

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,
GradientBoosting,0.409,0.646,0.704,0.597,0.876
GradientBoosting_t,0.409,0.646,0.704,0.597,0.876
LogisticRegression,0.290,0.510,0.462,0.568,0.772
RandomForestClassifier,0.950,NaN,0.000,0.000,0.865


GridSearchCV отобрал параметры,которые и так были использованы ранее. Экономическая эффективность GradientBoosting не изменилась.